In [1]:
from IPython import get_ipython
from IPython.display import display
# %%
import requests

lat = 40.7128
lon = -74.0060
api_key = "487b2ec534d2da96be1e7df58b057525"

url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
response = requests.get(url)

print(response.status_code)
print(response.json())

# %%
import requests # This libraray helps us to fetch data from API
import pandas as pd #for handling and analysing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #to split data into training and testing sets
from sklearn.preprocessing import LabelEncoder #to convert catogerical data into numericals values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and
from sklearn.metrics import mean_squared_error #to measure the accuracy of our predictions
from datetime import datetime, timedelta #to handle date and time
import pytz
# %%
API_KEY = "487b2ec534d2da96be1e7df58b057525"
BASE_URL = 'https://api.openweathermap.org/data/2.5/'
# %%

def get_current_weather (city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #construct the API, correcting the f-string formatting
  response = requests.get(url) # send the get request to API
  data = response.json()
  return { 
          'city': data['name'],
          'current_temp': round(data['main']['temp']),
          'feels_like': round(data['main']['feels_like']),
          'temp_min': round(data['main']['temp_min']),
          'temp_max': round(data['main']['temp_max']),
          'humidity': round (data['main'] ['humidity']),
          'description': data['weather'][0]['description'],
          'country': data['sys']['country'],
          'wind_gust_dir': data['wind']['deg'],
          'pressure': data['main']['pressure'],
          'Wind_GustSpeed': data['wind']['speed']
  }

# %%
def read_historical_data(filename):
  df = pd.read_csv(filename) #load csv file into dataFrame
  df = df.dropna () #remove rows wit missing values
  df = df.drop_duplicates() # Add parentheses to call the drop_duplicates method
  return df
# %%
def prepare_data(data):
  le = LabelEncoder() #create a LabelEncoder instance, removing the dataframe 'data' from the constructor
  # Check if 'WindGustDir' and 'RainTomorrow' columns exist
  if 'WindGustDir' in data.columns:
      data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  if 'RainTomorrow' in data.columns:
      data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])
  #define the feature variable and target variables
  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] #feature variables
  # Check if 'Rain Tomorrow' column exists
  if 'Rain Tomorrow' in data.columns:
      y = data['Rain Tomorrow'] #target variable
  else:
      y = None  # Or handle the case where the column doesn't exist
  
  return X, y, le # Correcting 'x' to 'X' to match the variable name used in the function
# %%
def train_rain_model(x, y):
  X_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier (n_estimators=100, random_state=42)
  model.fit(X_train, y_train) #train the model
  y_pred = model.predict(X_test) #to make predictions on test set

  print("Mean Squared Error for Rain Model")
  print(mean_squared_error(y_test, y_pred))
  return model
  
# %%
def prepare_regression_data(data, feature):
  X, y = [], [] #initialize list for feature and target values, Correcting 'x' to 'X'
  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i]) # Correcting 'x' to 'X'
    y.append(data[feature].iloc[i+1])
  X = np.array(X).reshape(-1, 1) # Correcting 'x' to 'X'
  y = np.array(y)
  return X, y # Correcting 'x' to 'X'
# %%
def train_regression_model(x, y):
  model = RandomForestRegressor (n_estimators=100, random_state=42)
  model.fit(x, y)
  return model
# %%
def predict_future (model, current_value):
  predictions = [current_value] # Initialize predictions with current_value
  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions [1:]
# %%
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)
#load historical data
  historical_data = read_historical_data('weather.csv')
#prepae and train the rain prediction model
  x, y, le = prepare_data(historical_data)
  # Check if y is not None before training the model
  if y is not None:
      rain_model = train_rain_model(x, y)
  else:
      print("RainTomorrow column not found in historical data. Skipping rain prediction.")
      rain_model = None  # Or handle the case where the column doesn't exist

  #map wind direction to campass points
  wind_deg = current_weather ['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)
  ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
  
  current_data = {
    'MinTemp': current_weather['temp_min'],
    'MaxTemp': current_weather['temp_max'],
    'WindGustSpeed': current_weather['Wind_GustSpeed'],
    'Humidity': current_weather['humidity'],
    'Pressure': current_weather['pressure'],
    'Temp': current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data]) 

  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')
  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])
#prepare time for future predictions
  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]
#Display results
  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}")
  print(f"Feels Like: {current_weather['feels_like']}")
  print(f"Minimum Temperature: {current_weather['temp_min']}")
  # Check if 'temp min' key exists in current_weather
  if 'temp min' in current_weather:
      print(f"Minimum Temperature: {current_weather['temp min']}")
  else:
      print("Minimum Temperature (temp min) not found in current weather data.")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  # Check if rain_model is not None before making rain prediction
  if rain_model is not None:
      # Assuming you have some logic to get rain_prediction (replace with your actual logic)
      # For example, rain_prediction = rain_model.predict(...)
      rain_prediction = False  # Placeholder, replace with your prediction logic
      print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")
  else:
      print("Rain prediction skipped due to missing RainTomorrow column in historical data.")
  print("\nFuture Temperature Predictions:")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions:")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()

200
{'coord': {'lon': -74.006, 'lat': 40.7128}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 279.38, 'feels_like': 276.08, 'temp_min': 279.38, 'temp_max': 279.38, 'pressure': 1023, 'humidity': 71, 'sea_level': 1023, 'grnd_level': 1022}, 'visibility': 10000, 'wind': {'speed': 4.89, 'deg': 109, 'gust': 8.35}, 'clouds': {'all': 100}, 'dt': 1744346851, 'sys': {'country': 'US', 'sunrise': 1744366973, 'sunset': 1744414239}, 'timezone': -14400, 'id': 5128581, 'name': 'New York', 'cod': 200}


Enter any city name:  Mumbai


RainTomorrow column not found in historical data. Skipping rain prediction.
City: Mumbai, IN
Current Temperature: 28
Feels Like: 30
Minimum Temperature: 28
Minimum Temperature (temp min) not found in current weather data.
Humidity: 69%
Weather Prediction: clear sky
Rain prediction skipped due to missing RainTomorrow column in historical data.

Future Temperature Predictions:
10:00: 27.7°C
11:00: 22.8°C
12:00: 25.4°C
13:00: 24.7°C
14:00: 23.4°C

Future Humidity Predictions:
10:00: 61.3%
11:00: 58.5%
12:00: 54.2%
13:00: 58.1%
14:00: 54.2%


In [3]:
import streamlit as st
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt

API_KEY = "487b2ec534d2da96be1e7df58b057525"
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'Wind_GustSpeed': data['wind']['speed']
    }

def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna().drop_duplicates()
    return df

def prepare_data(data):
    le = LabelEncoder()
    if 'WindGustDir' in data.columns:
        data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    return X, le

def prepare_regression_data(data, feature):
    X, y = [], []
    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])
    return np.array(X).reshape(-1, 1), np.array(y)

def train_regression_model(x, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x, y)
    return model

def predict_future(model, current_value):
    predictions = [current_value]
    for _ in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]

def main():
    st.set_page_config(layout="wide", page_title="Dynamic Weather Forecast Dashboard")
    st.markdown("<h1 style='text-align: center;'>🌤️ Weather Forecast Dashboard</h1>", unsafe_allow_html=True)

    city = st.text_input("Enter a city", "London")

    if city:
        current_weather = get_current_weather(city)
        historical_data = read_historical_data("weather.csv")
        x, le = prepare_data(historical_data)

        # Wind direction mapping
        wind_deg = current_weather['wind_gust_dir'] % 360
        compass_points = [
            ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
            ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
            ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
            ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
            ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
            ("NNW", 326.25, 348.75)
        ]
        compass_direction = next((point for point, start, end in compass_points if start <= wind_deg < end), "N")
        compass_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

        X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
        X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

        temp_model = train_regression_model(X_temp, y_temp)
        hum_model = train_regression_model(X_hum, y_hum)

        future_temp = predict_future(temp_model, current_weather['temp_min'])
        future_humidity = predict_future(hum_model, current_weather['humidity'])

        timezone = pytz.timezone('Asia/Kolkata')
        now = datetime.now(timezone)
        next_hour = now + timedelta(hours=1)
        next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
        future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

        col1, col2 = st.columns([1, 3])

        with col1:
            st.metric("🌡️ Temperature", f"{current_weather['current_temp']}°C", f"Feels like {current_weather['feels_like']}°C")
            st.metric("💧 Humidity", f"{current_weather['humidity']}%")
            st.metric("☁️ Description", current_weather['description'].capitalize())
            st.metric("🌬️ Wind", f"{current_weather['Wind_GustSpeed']} m/s ({compass_direction})")
            st.metric("📊 Pressure", f"{current_weather['pressure']} mb")

        with col2:
            st.subheader("📈 Next Hours Forecast")
            df_future = pd.DataFrame({
                "Time": future_times,
                "Temperature (°C)": future_temp,
                "Humidity (%)": future_humidity
            })

            st.line_chart(df_future.set_index("Time"))

if __name__ == "__main__":
    main()


2025-04-11 10:28:35.130 
  command:

    streamlit run C:\an\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-11 10:28:35.141 Session state does not function when running a script without `streamlit run`
